In [1]:
import numpy as np 
import pandas as pd 
from os import listdir
from os.path import isfile, join


In [2]:
# list(csvfiles)
# subjectIDs.values.flatten().astype(str)


In [12]:
mypath = "/projectnb/skiran/Isaac/data_for_saurav/"
subjectIDs = pd.read_csv(mypath+"Subject_IDs.csv", header = None)

filenames = listdir(mypath)
csvfiles = [ filename for filename in filenames if filename.endswith( ".csv" ) ]
csvfiles.remove('Subject_IDs.csv')
csvfiles = sorted(csvfiles)
pdf = {"patientIDs" : list(subjectIDs.values.flatten().astype(str)),
       "alias" : list(csvfiles)
      }

pdf_alias = pd.DataFrame(pdf)


df = pd.read_excel("/projectnb/skiran/saurav/Fall-2022/RS" + "/compiled_dataset_RSbivariate_without_controls_v7.xlsx", header = [0,1])
outputs = df[("behavioral", "wab_aq_bd")]

pdf = { "patientIDs": list(df[("participant", "participant")]),
        "scores" : list(outputs) 
      }
pdf_scores = pd.DataFrame(pdf)

print("pdf alias shape = ", pdf_alias.shape, " -- pdf score shape = ", pdf_scores.shape)


pdf alias shape =  (65, 2)  -- pdf score shape =  (55, 2)


In [16]:

corr_data = []
scores = []
for patient, score in pdf_scores.values:
    if patient != "BU29c07":
#         print("patient = ", patient, " -- scores = ", score, " -- alias = ", list(pdf_alias[pdf_alias["patientIDs"] == patient]["alias"])[0])
        fname = mypath  + list(pdf_alias[pdf_alias["patientIDs"] == patient]["alias"])[0]
        scores.append(score)
        df = pd.read_csv(fname)
        columns = df.columns
        df.index = columns
        corr_data.append(df)
    else:
#         print("patient = ", patient, " -- scores = ", score, " -- alias = ", list(pdf_alias[pdf_alias["patientIDs"] == patient+"NU"]["alias"])[0])
        fname = mypath  + list(pdf_alias[pdf_alias["patientIDs"] == patient+"NU"]["alias"])[0]
        df = pd.read_csv(fname)
        columns = df.columns
        df.index = columns
        scores.append(score)
        corr_data.append(df)
        

correlation_data = corr_data
for index,data in enumerate(correlation_data):
    df = data
    df = df.where(np.triu(np.ones(df.shape),k=1).astype(np.bool))
    df = df.stack().reset_index()
    df.columns = ['Row','Column','Value']
    
    df["index"] = df["Row"] + "-vs-" + df["Column"]
    correlation_data[index] = df["Value"].values #data.values[np.triu_indices(48, k = 1)]
    all_features = df["index"]
correlation_data[0]

/scratch/131859.1.onrcc-m256/ipykernel_10267/3420246208.py:25: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df = df.where(np.triu(np.ones(df.shape),k=1).astype(np.bool))


array([0.54318573, 0.49769502, 0.15269705, ..., 0.54129219, 0.45752985,
       0.39583786])

In [18]:
correlation_data = np.array(correlation_data)
correlation_data


array([[ 0.54318573,  0.49769502,  0.15269705, ...,  0.54129219,
         0.45752985,  0.39583786],
       [ 0.31337034,  0.60173758,  0.33918601, ...,  0.41270039,
         0.2502227 ,  0.26021853],
       [ 0.39162063,  0.61841994,  0.0276738 , ...,  0.34259815,
         0.12261526,  0.38368191],
       ...,
       [ 0.35124405,  0.62184308,  0.27302733, ...,  0.26199443,
         0.12609409,  0.64633376],
       [ 0.33181602,  0.41715246, -0.01356422, ...,  0.24050942,
         0.45911877,  0.36147037],
       [ 0.48555447,  0.42244135,  0.25793505, ...,  0.27152909,
         0.61172267,  0.28565473]])

In [23]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

regr =  SVR(C=1.0, epsilon=0.2)#make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(correlation_data, scores)

SVR(epsilon=0.2)

In [166]:
correlation_data = pd.DataFrame(correlation_data, columns = all_features)
correlation_data

data = correlation_data.values 
data = np.hstack((data,np.array(outputs).reshape(55,1)))

data = pd.DataFrame(data)

corrs = data.corr().abs()
importances = corrs.values[-1][:-1]
importances

top_columns = np.argpartition(importances, -10)[-10:]

columns = list(correlation_data.columns[top_columns])
imps = importances[top_columns]

df = {
    "columns" : columns,
    "importances" : imps
}

df = pd.DataFrame(df)
df

,columns,importances
0,L_INS-vs-L_STG,0.387281
1,L_MTG-vs-R_IFGop,0.395146
2,L_IFGtri-vs-L_ITG,0.397811
3,L_STG-vs-R_STG,0.458890
4,L_SMG-vs-R_SMG,0.399784
5,L_ITG-vs-R_MFG,0.432364
6,L_IFGorb-vs-L_MTG,0.547716
7,L_SFGmedial-vs-R_midTP,0.472901
8,L_IFGtri-vs-L_MTG,0.448727
9,L_MFG-vs-R_OFGmedial,0.427919
